In [1]:
import os
import sys
sys.path.append(os.chdir('../../'))

In [18]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import datetime

import matplotlib.pyplot as plt
import plotly.plotly as ply
import plotly.graph_objs as go
import src.utils.graph as graph
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

tqdm.pandas()
pd.options.display.max_columns = None

In [3]:
import src.data as data
import src.utility as utility

In [4]:
sensors = data.sensors()

s1 = data.speeds('train').merge(sensors, how='left')
#s2 = data.speeds('test').merge(sensors, how='left')

e1 = data.events('train')
#e2 = data.events('test')

caching sensors

sensors loaded in: 0.0097 s

shape of the dataframe is: (2171, 5)
caching resources/dataset/originals/speeds_train.csv.gz
caching resources/dataset/preprocessed/events_train.csv.gz


In [5]:
joined = utility.merge_speed_events(s1, e1)

In [6]:
joined.head()

,KEY,DATETIME_UTC,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,KEY_2,EMERGENCY_LANE,LANES,ROAD_TYPE,event_index,START_DATETIME_UTC,END_DATETIME_UTC,EVENT_DETAIL,EVENT_TYPE,KM_END,KM_START,KM_EVENT,step_duration
0,0,2018-09-01 01:00:00,443,97.889000,1.144099,97.080,98.698,2,0_443,1,2,1,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2018-09-01 03:15:00,443,154.788667,16.287509,137.530,169.890,3,0_443,1,2,1,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2018-09-01 12:45:00,443,136.091778,10.759666,124.586,163.418,18,0_443,1,2,1,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2018-09-01 19:15:00,443,136.916276,15.089653,114.878,169.890,58,0_443,1,2,1,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2018-09-02 09:45:00,443,135.476385,23.125778,82.518,169.890,26,0_443,1,2,1,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
start_events = joined[joined.event_index.notnull()].groupby(['KEY_2','event_index']).first()

In [14]:
start_events.head()

KEY        DATETIME_UTC   KM   SPEED_AVG   SPEED_SD  \
KEY_2 event_index                                                        
0_337 16789.0        0 2018-09-05 19:00:00  337  201.772145  49.993015   
      17038.0        0 2018-09-15 11:45:00  337  217.911338  30.139167   
      17088.0        0 2018-09-17 16:15:00  337  211.920078  29.666366   
      17208.0        0 2018-09-20 16:30:00  337  173.413308  33.294010   
      17460.0        0 2018-10-01 17:15:00  337  201.965352  39.702932   

                   SPEED_MIN  SPEED_MAX  N_VEHICLES  EMERGENCY_LANE  LANES  \
KEY_2 event_index                                                            
0_337 16789.0         59.866    312.274         193               1      2   
      17038.0         61.484    288.004         287               1      2   
      17088.0         59.866    300.948         256               1      2   
      17208.0         59.866    236.228         214               1      2   
      17460.0         59.866    286.386         216               1      2   

                   ROAD_TYPE  START_DATETIME_UTC    END_DATETIME_UTC  \
KEY_2 event_index                                                      
0_337 16789.0              1 2018-09-05 18:15:00 2018-09-05 19:00:00   
      17038.0              1 2018-09-15 11:15:00 2018-09-15 11:45:00   
      17088.0              1 2018-09-17 16:15:00 2018-09-17 16:45:00   
      17208.0              1 2018-09-20 16:30:00 2018-09-20 18:45:00   
      17460.0              1 2018-10-01 17:00:00 2018-10-01 17:45:00   

                   EVENT_DETAIL               EVENT_TYPE  KM_END  KM_START  \
KEY_2 event_index                                                            
0_337 16789.0              11.0        Veicolo_in_avaria   337.0     331.0   
      17038.0               4.0  Ostacolo_in_carreggiata   337.0     331.0   
      17088.0               4.0  Ostacolo_in_carreggiata   337.0     331.0   
      17208.0               6.0       Gestione_viabilita   337.0     336.0   
      17460.0               6.0       Gestione_viabilita   337.0     336.0   

                   KM_EVENT  step_duration  
KEY_2 event_index                           
0_337 16789.0           NaN            4.0  
      17038.0           NaN            3.0  
      17088.0           NaN            3.0  
      17208.0           NaN           10.0  
      17460.0           NaN            4.0

In [21]:
start_events['step_before'] = start_events.DATETIME_UTC - datetime.timedelta(minutes=15)

In [28]:
diff_start_events = start_events.reset_index().merge(s1.drop(['EMERGENCY_LANE','LANES','ROAD_TYPE'],axis=1), how='left', left_on=['KEY','KM','step_before'], right_on=['KEY','KM','DATETIME_UTC'])

In [38]:
diff_start_events = diff_start_events.rename(columns={'SPEED_AVG_x':'SPEED_AVG_dopo', 'SPEED_AVG_y':'SPEED_AVG_prima'})
diff_start_events = diff_start_events.rename(columns={'SPEED_SD_x':'SPEED_SD_dopo', 'SPEED_SD_y':'SPEED_SD_prima'})
diff_start_events = diff_start_events.rename(columns={'SPEED_MIN_x':'SPEED_MIN_dopo', 'SPEED_MIN_y':'SPEED_MIN_prima'})
diff_start_events = diff_start_events.rename(columns={'SPEED_MAX_x':'SPEED_MAX_dopo', 'SPEED_MAX_y':'SPEED_MAX_prima'})
diff_start_events = diff_start_events.rename(columns={'N_VEHICLES_x':'N_VEHICLES_dopo', 'N_VEHICLES_y':'N_VEHICLES_prima'})
diff_start_events.head()

,KEY_2_x,event_index,KEY,DATETIME_UTC_x,KM,SPEED_AVG_dopo,SPEED_SD_dopo,SPEED_MIN_dopo,SPEED_MAX_dopo,N_VEHICLES_dopo,EMERGENCY_LANE,LANES,ROAD_TYPE,START_DATETIME_UTC,END_DATETIME_UTC,EVENT_DETAIL,EVENT_TYPE,KM_END,KM_START,KM_EVENT,step_duration,step_before,DATETIME_UTC_y,SPEED_AVG_prima,SPEED_SD_prima,SPEED_MIN_prima,SPEED_MAX_prima,N_VEHICLES_prima,KEY_2_y,speed_avg_diff
0,0_337,16789.0,0,2018-09-05 19:00:00,337,201.772145,49.993015,59.866,312.274,193,1,2,1,2018-09-05 18:15:00,2018-09-05 19:00:00,11.0,Veicolo_in_avaria,337.0,331.0,NaN,4.0,2018-09-05 18:45:00,2018-09-05 18:45:00,204.775942,43.065888,59.866,294.476,139.0,0_337,-3.003797
1,0_337,17038.0,0,2018-09-15 11:45:00,337,217.911338,30.139167,61.484,288.004,287,1,2,1,2018-09-15 11:15:00,2018-09-15 11:45:00,4.0,Ostacolo_in_carreggiata,337.0,331.0,NaN,3.0,2018-09-15 11:30:00,2018-09-15 11:30:00,215.581482,31.902129,59.866,330.072,309.0,0_337,2.329856
2,0_337,17088.0,0,2018-09-17 16:15:00,337,211.920078,29.666366,59.866,300.948,256,1,2,1,2018-09-17 16:15:00,2018-09-17 16:45:00,4.0,Ostacolo_in_carreggiata,337.0,331.0,NaN,3.0,2018-09-17 16:00:00,2018-09-17 16:00:00,213.911089,39.193833,59.866,283.150,169.0,0_337,-1.991011
3,0_337,17208.0,0,2018-09-20 16:30:00,337,173.413308,33.294010,59.866,236.228,214,1,2,1,2018-09-20 16:30:00,2018-09-20 18:45:00,6.0,Gestione_viabilita,337.0,336.0,NaN,10.0,2018-09-20 16:15:00,2018-09-20 16:15:00,177.033031,23.730896,59.866,257.262,258.0,0_337,-3.619723
4,0_337,17460.0,0,2018-10-01 17:15:00,337,201.965352,39.702932,59.866,286.386,216,1,2,1,2018-10-01 17:00:00,2018-10-01 17:45:00,6.0,Gestione_viabilita,337.0,336.0,NaN,4.0,2018-10-01 17:00:00,2018-10-01 17:00:00,208.912353,38.880712,59.866,352.724,238.0,0_337,-6.947001


In [40]:
diff_start_events['speed_avg_diff'] = diff_start_events['SPEED_AVG_dopo'] - diff_start_events['SPEED_AVG_prima']
diff_start_events['speed_sd_diff'] = diff_start_events['SPEED_SD_dopo'] - diff_start_events['SPEED_SD_prima']
diff_start_events['speed_min_diff'] = diff_start_events['SPEED_MIN_dopo'] - diff_start_events['SPEED_MIN_prima']
diff_start_events['speed_max_diff'] = diff_start_events['SPEED_MAX_dopo'] - diff_start_events['SPEED_MAX_prima']
diff_start_events['n_vehicles_diff'] = diff_start_events['N_VEHICLES_dopo'] - diff_start_events['N_VEHICLES_prima']

In [41]:
diff_start_events[['EVENT_TYPE','speed_avg_diff','speed_sd_diff','speed_min_diff','speed_max_diff','n_vehicles_diff']].groupby('EVENT_TYPE').mean()

,speed_avg_diff,speed_sd_diff,speed_min_diff,speed_max_diff,n_vehicles_diff
EVENT_TYPE,,,,,
Allarme,1.092050,-0.515786,2.294618,-3.971455,0.090909
Barriere,-2.628026,-2.724998,-2.022500,-10.517000,-6.300000
Calamita_naturale,2.570122,-3.796751,14.673586,-1.785379,6.793103
Gestione_viabilita,0.470544,-0.057648,-0.064720,0.504816,0.368000
Manutenzione_opere_in_verde,-0.653729,-0.317261,-2.427000,-1.866923,4.057692
Meteo,-3.177306,-0.974971,-3.158952,-4.237619,-10.571429
Opera_arte,2.111584,-4.070666,0.404500,-18.539583,-6.250000
Ostacolo_in_carreggiata,-0.341778,0.312343,1.134573,2.318476,-0.835366
Pavimentazione,0.195942,-0.613817,9.367368,0.915447,-5.763158
